# Gradio Interface
# Tarea#6

In [1]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings
import pandas as pd
import tiktoken
import openai
import ast
import os

In [2]:
openai.api_key = "sk-proj-bFgOxkMI67b3GoJiiErBT3BlbkFJdaDKma1UOTCVPwJsqRAr" # YOUR API KEY
EMBEDDING_MODEL_NAME = "text-embedding-ada-002" #Nombre de modelo creado en el paso 1
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct" #Nombre de modelo creado en paso 3 y 4
BATCH_SIZE = 128

In [7]:
df = pd.read_csv('embeddings_txt.csv') #Reemplazar por nombre de csv
df.embeddings = df.embeddings.apply(ast.literal_eval) 

In [8]:
def get_rows_sorted_by_relevance(question, df): 
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings( #!
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True) #!
    return df_copy

In [9]:
get_rows_sorted_by_relevance('hello my friend', df) #!

,Unnamed: 0,text,embeddings,distances
49,49,Topics,"[-0.011941291391849518, -0.012782999314367771,...",0.216218
174,174,Frequently Asked Queestions (FAQs),"[0.018329203128814697, -0.009138453751802444, ...",0.247824
243,243,Frequently Asked Queestions (FAQs),"[0.018303731456398964, -0.009151865728199482, ...",0.247838
1028,1028,Agnosia can compromise quality of life.,"[0.0238259956240654, -0.0030998445581644773, 0...",0.258042
280,280,PLS is not fatal. There is no cure and the pro...,"[0.006643248256295919, 0.0010966863483190536, ...",0.264773
...,...,...,...,...
1985,1985,Other types of diabetes have a variety of poss...,"[-0.021894806995987892, -0.024371536448597908,...",0.335929
1886,1886,Other types of diabetes have a variety of poss...,"[-0.021894806995987892, -0.024371536448597908,...",0.335929
1704,1704,Other types of diabetes have a variety of poss...,"[-0.021894806995987892, -0.024371536448597908,...",0.335929
1887,1887,Other types of diabetes have a variety of poss...,"[-0.021893849596381187, -0.024408770725131035,...",0.335963


In [10]:
def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base") #!
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [11]:
print(create_prompt('hello my friend', df, max_token_count=1800)) #!


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

Topics

###

Frequently Asked Queestions (FAQs)

###

Frequently Asked Queestions (FAQs)

###

Agnosia can compromise quality of life.

###

PLS is not fatal. There is no cure and the progression of symptoms varies. Some people may retain the ability to walk without assistance, but others eventually require wheelchairs, canes, or other assistive devices.

###

Hepatitis B is called a silent killer because many people have no symptoms, so the disease often progresses unnoticed for years. Unfortunately, many people first learn they have chronic hepatitis B when they develop symptoms of severe liver damage, which include                - yellowish eyes and skin, called jaundice  - a swollen stomach or ankles  - tiredness  - nausea  - weakness  - loss of appetite  - weight loss  - spiderlike blood vessels, called spider angiomas, that develop on the sk

In [12]:
def answer_question(question, df, max_prompt_tokens=1800, max_answer_tokens=600):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

In [13]:
print(answer_question('How to prevent Parasites - Cysticercosis', df)) #!

To prevent parasites, specifically cysticercosis, you should follow the precautions listed in the context. These include washing your hands with soap and warm water, teaching children the importance of hand washing, and avoiding contaminated food and water.


In [2]:
import openai
import gradio as gr


def predict(message, history):
    global df
    return answer_question(message, df)
gr.ChatInterface(predict).launch(share=True)

Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
